# Case Study: American Community Survey Data

Let's look at an example of how we might utilize these workflows on an actual dataset, specifically <a href="https://data.census.gov/table/ACSST5Y2022.S1501?t=Education:Educational Attainment&g=050XX00US18141,18141$8600000&moe=false">educational outcome attribute data from the American Community Survey</a>
  * [Link to download](https://raw.githubusercontent.com/kwaldenphd/elements-of-computing/main/book/data/ch5/data.csv)
  * *Note: I'm working with 2022's 5 year estimate, which I've renamed `data.csv`.*

In [ ]:
import pandas as pd, re # import statements
df = pd.read_csv("https://raw.githubusercontent.com/kwaldenphd/elements-of-computing/main/book/data/ch5/data.csv") # load data
df # show output

A rich dataset, but not in particularly usable shape at the moment.


## Step #1: Building an Outline

A good starting point is sketching out how you want the data to look at the end of reshaping operations. We can start thinking of this as the data processing version of pseudocode.

For this dataset, you might want something like....

| Area | Category | Variable | Gender | Value |
| --- | --- | --- | --- | --- |
| *geographic unit* | *category* | *specific field* | *gender breakdown* | *amount or measure* |

Based on where the data is now, some things we might need to do...
- Take the category rows, make them a new column, and create a multi-level index on that axis
- Break out the current column headers
- Make the updated column headers their own columns
- Probably relabel the columns to more user-friendly labels

Let's dig in!

## Step #2: Breaking out category and variable columns

Let's take a look at the first column.

In [ ]:
df.iloc[:, 0].unique() # inspect unique values in first column

A few things going on here, but the first thing we might want to do is remove the `\xa0` unicode character.

In [ ]:
df.replace(r"\xa0", '', regex=True, inplace=True) # remove unicode character
df.iloc[:, 0].unique() # inspect output

Then, we can use a couple conditional statements to add a `Category` column.
- First, we'll insert an empty column at the start of our DataFrame
- Then, we'll iterate over rows, test if the third cell is null (`NaN`)
- If that test is `True`, we'll add the category value to the first column

In [ ]:
df.insert(0, 'Category', '', True) # insert new column for category

for i, r in df.iterrows(): # iterate over rows, insert new column value
  if pd.isnull(r.iloc[2]) == True: # test if third cell is a NaN value
    df['Category'].iloc[i] = r.iloc[1] # if null test if True, add that value to a new column
  else: # otherwise, keep iterating over rows
    continue

df # inspect output

Making progress! Let's break down the pseudocode version of next steps:
- Now that we have a `Category` column, we can filter out the `NaN` rows.
- Then, we can forward fill or fill down the `Category` column values so there's a connection between the category and variable lable

A couple last steps before we think about the next reshaping operation:
- Rename the second column
- Reset the index (after filtering)

In [ ]:
df = df.mask(df == '') # mask empty cells with NaN
df.iloc[:, 0].ffill(inplace=True) # foward fill category column
df.dropna(inplace=True) # drop rows with NA
df.columns.values[1] = 'Variable' # rename column
df.reset_index(inplace=True, drop=True) # reset index
df # show output

## Step #3: Melting Variable Labels

Now that we have a `Category` and `Variable` columns, let's melt this dataframe so the existing column labels become their own column. We'll use these two columns as `id_vars`.

We can also rename the columns after this operation.

In [ ]:
df = pd.melt(df, id_vars=['Category', 'Variable']) # melt everything but the first two columns
df.columns = ['category', 'variable', 'metadata', 'value']
df # inspect output

## Step #4: Splitting multi-value column

If we take a closer look at the `metadata` column, there are three key pieces of information:
- Geographic scope (county, zip code, etc)
- Gender scope (total, male, female)
- Measure type (all estimates, so we'll eventually drop this value)

We'll use `!!` as a separator for regular expression string methods.
- [Click here](https://colab.research.google.com/drive/1Rd9KMoJ2AdtwVMNXwibakUbMGxz5Z6nM?usp=sharing#scrollTo=c1TKLsmiSS9N) for a Jupyter Notebook that provides a deep dive into regular expressions (regex) and string methods in Python.

In [ ]:
df[['area', 'gender', 'measure']] = df['metadata'].str.split('!!', expand=True) # split metadata column using separator
df # inspect output

## Step #5: Subsetting & Filtering

Last but not least, we might want to subset our `DataFrame` for meaningful columns, and remove rows with `NaN` values.

In [ ]:
df = df[['area', 'category', 'variable', 'gender', 'value']] # subset columns
df = df[df['value'].notnull()] # remove rows with NaN in value
df = df.reset_index(drop=True) # reset index
df['area'] = df['area'].str.replace("ZCTA5 ", "") # clean up area column to be able to join on zip code
df # show output

There's more we might want to do with processing or filtering this data, but a solid start to programmatically reshape this data.

## Wrap Up

Voila! Let's see all of those steps together:

In [ ]:
import pandas as pd, re # import statements
df = pd.read_csv("https://raw.githubusercontent.com/kwaldenphd/elements-of-computing/main/book/data/ch5/data.csv") # load data

df.replace(r"\xa0", '', regex=True, inplace=True) # remove unicode character

df.insert(0, 'Category', '', True) # insert new column for category

for i, r in df.iterrows(): # iterate over rows, insert new column value
  if pd.isnull(r.iloc[2]) == True: # test if third cell is a NaN value
    df['Category'].iloc[i] = r.iloc[1] # if null test if True, add that value to a new column
  else: # otherwise, keep iterating over rows
    continue

df = df.mask(df == '') # mask empty cells with NaN
df.iloc[:, 0].ffill(inplace=True) # foward fill category column
df.dropna(inplace=True) # drop rows with NA
df.columns.values[1] = 'Variable' # rename column
df.reset_index(inplace=True, drop=True) # reset index

df = pd.melt(df, id_vars=['Category', 'Variable']) # melt everything but the first two columns
df.columns = ['category', 'variable', 'metadata', 'value']
df[['area', 'gender', 'measure']] = df['metadata'].str.split('!!', expand=True) # split metadata column using separator

df = df[['area', 'category', 'variable', 'gender', 'value']] # subset columns
df = df[df['value'].notnull()] # remove rows with NaN in value
df = df.reset_index(drop=True) # reset index
df['area'] = df['area'].str.replace("ZCTA5 ", "") # clean up area column to be able to join on zip code
df.to_csv("output.csv", index=False) # write output to csv
df # show output